## 1. Introduction/Business Problem

Chicago reduced a murder rate in 2018. The municipal authorities want to keep this trend. 
They outsourced to prepare a report which is to identify crime patterns in Chicago. 

The report should provide answers for below questions including difference between overall crimes and homicides:
* When crimes are committed? 
* Which community areas are the most dangerous? (crimes per 1000 inhabitants)
* Does police beats are properly located?
* What are the nearest venues where homicides are committed? (Foursquare location)


## 2. Data

The sources of data are official  websites: https://data.cityofchicago.org and https://www.chicago.gov

Finally two tables were created:
* total - containing all crimes
* homicide - containing this kind of crime

Both tables consist of 15 columns which determine:
* time of crime - Season, Part of the day
* place of crime - Block, Location description,Longitude, Latitude, Community Area, Community Area Description, 
* crime classification - Case number, Primary Type, Arrest
* police assignment - Beat, District

In [1]:
import pandas as pd 
pd.set_option('display.max_columns', None) # Show all columns
pd.set_option('display.max_rows', None) # Show all columns

# Download crime table - more information on website : https://data.cityofchicago.org/Public-Safety/Crimes-2018/3i3m-jwuy

crime = pd.read_csv('https://data.cityofchicago.org/api/views/3i3m-jwuy/rows.csv?accessType=DOWNLOAD')

crime = crime.drop(['ID','IUCR','Ward','FBI Code','X Coordinate','Y Coordinate','Year','Updated On','Location','Historical Wards 2003-2015',
              'Zip Codes','Community Areas','Census Tracts','Wards','Boundaries - ZIP Codes','Police Districts','Police Beats','Domestic'],axis=1) 

crime['Location Description'].fillna("OTHER",inplace=True) #populate missing Location Descriptions

crime.head()

,Case Number,Date,Block,Primary Type,Description,Location Description,Arrest,Beat,District,Community Area,Latitude,Longitude
0,JC104662,12/31/2018 11:59:00 PM,112XX S SACRAMENTO AVE,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,2211,22,74.0,41.689079,-87.696064
1,JC110056,12/31/2018 11:59:00 PM,013XX W 72ND ST,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,OTHER,False,734,7,67.0,41.763181,-87.657709
2,JC100043,12/31/2018 11:57:00 PM,084XX S SANGAMON ST,CRIMINAL DAMAGE,TO PROPERTY,APARTMENT,False,613,6,71.0,41.740521,-87.647391
3,JC100006,12/31/2018 11:56:00 PM,018XX S ALLPORT ST,BATTERY,AGG: HANDS/FIST/FEET NO/MINOR INJURY,OTHER,True,1233,12,31.0,41.857068,-87.657625
4,JC100031,12/31/2018 11:55:00 PM,078XX S SANGAMON ST,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,621,6,71.0,41.751914,-87.647717


In [2]:
crime.shape

(267520, 12)

In [3]:
#Download population table

population = pd.read_csv('https://raw.githubusercontent.com/piotr-ek7/Coursera_Capstone/master/Population.csv',error_bad_lines=False) #source: https://www.chicago.gov/city/en/depts/dcd/supp_info/community_area_2000and2010censuspopulationcomparisons.html
population.columns =['Community Area','Community Area Description','Community Area Population'] #rename columns
population.head()

,Community Area,Community Area Description,Community Area Population
0,1,Rogers Park,54991
1,2,West Ridge,71942
2,3,Uptown,56362
3,4,Lincoln Square,39493
4,5,North Center,31867


In [4]:
df = pd.merge(crime,population,on='Community Area',how="left") #join crime and population tables
df.head()

,Case Number,Date,Block,Primary Type,Description,Location Description,Arrest,Beat,District,Community Area,Latitude,Longitude,Community Area Description,Community Area Population
0,JC104662,12/31/2018 11:59:00 PM,112XX S SACRAMENTO AVE,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,2211,22,74.0,41.689079,-87.696064,Mount Greenwood,19093.0
1,JC110056,12/31/2018 11:59:00 PM,013XX W 72ND ST,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,OTHER,False,734,7,67.0,41.763181,-87.657709,West Englewood,35505.0
2,JC100043,12/31/2018 11:57:00 PM,084XX S SANGAMON ST,CRIMINAL DAMAGE,TO PROPERTY,APARTMENT,False,613,6,71.0,41.740521,-87.647391,Auburn Gresham,48743.0
3,JC100006,12/31/2018 11:56:00 PM,018XX S ALLPORT ST,BATTERY,AGG: HANDS/FIST/FEET NO/MINOR INJURY,OTHER,True,1233,12,31.0,41.857068,-87.657625,Lower West Side,35769.0
4,JC100031,12/31/2018 11:55:00 PM,078XX S SANGAMON ST,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,621,6,71.0,41.751914,-87.647717,Auburn Gresham,48743.0


In [5]:
#Assignng a given season based on Date
month = df['Date'].str[:2].astype('int32') #extract month
season=[]

for m in month:
    if m in range(3,6): season.append('Spring')
    elif m in range(6,9): season.append('Summer')
    elif m in range(9,12): season.append('Fall')
    else: season.append('Winter')

season = pd.DataFrame(season,columns=["Season"])
df = pd.merge(df,season,left_index=True,right_index=True)
df.head()

,Case Number,Date,Block,Primary Type,Description,Location Description,Arrest,Beat,District,Community Area,Latitude,Longitude,Community Area Description,Community Area Population,Season
0,JC104662,12/31/2018 11:59:00 PM,112XX S SACRAMENTO AVE,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,2211,22,74.0,41.689079,-87.696064,Mount Greenwood,19093.0,Winter
1,JC110056,12/31/2018 11:59:00 PM,013XX W 72ND ST,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,OTHER,False,734,7,67.0,41.763181,-87.657709,West Englewood,35505.0,Winter
2,JC100043,12/31/2018 11:57:00 PM,084XX S SANGAMON ST,CRIMINAL DAMAGE,TO PROPERTY,APARTMENT,False,613,6,71.0,41.740521,-87.647391,Auburn Gresham,48743.0,Winter
3,JC100006,12/31/2018 11:56:00 PM,018XX S ALLPORT ST,BATTERY,AGG: HANDS/FIST/FEET NO/MINOR INJURY,OTHER,True,1233,12,31.0,41.857068,-87.657625,Lower West Side,35769.0,Winter
4,JC100031,12/31/2018 11:55:00 PM,078XX S SANGAMON ST,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,621,6,71.0,41.751914,-87.647717,Auburn Gresham,48743.0,Winter


In [6]:
#Assigning the part of day based on Date

hour = df['Date'].str[11:13] + df['Date'].str[-2:] #extract time
hours = [int(h[:2])+12 if h[-2:]=="PM" else int(h[:2]) for h in hour] #covert to 24h format

part_of_day=[]

for p in hours:
    if p in range(5,12): part_of_day.append('Morning')
    elif p in range(12,17): part_of_day.append('Afternoon')
    elif p in range(17,21): part_of_day.append('Evening')
    else: part_of_day.append('Night')
        
part_of_day = pd.DataFrame(part_of_day,columns=["Part of the day"])
df = pd.merge(df,part_of_day,left_index=True,right_index=True) 
df.head()

,Case Number,Date,Block,Primary Type,Description,Location Description,Arrest,Beat,District,Community Area,Latitude,Longitude,Community Area Description,Community Area Population,Season,Part of the day
0,JC104662,12/31/2018 11:59:00 PM,112XX S SACRAMENTO AVE,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,2211,22,74.0,41.689079,-87.696064,Mount Greenwood,19093.0,Winter,Night
1,JC110056,12/31/2018 11:59:00 PM,013XX W 72ND ST,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,OTHER,False,734,7,67.0,41.763181,-87.657709,West Englewood,35505.0,Winter,Night
2,JC100043,12/31/2018 11:57:00 PM,084XX S SANGAMON ST,CRIMINAL DAMAGE,TO PROPERTY,APARTMENT,False,613,6,71.0,41.740521,-87.647391,Auburn Gresham,48743.0,Winter,Night
3,JC100006,12/31/2018 11:56:00 PM,018XX S ALLPORT ST,BATTERY,AGG: HANDS/FIST/FEET NO/MINOR INJURY,OTHER,True,1233,12,31.0,41.857068,-87.657625,Lower West Side,35769.0,Winter,Night
4,JC100031,12/31/2018 11:55:00 PM,078XX S SANGAMON ST,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,621,6,71.0,41.751914,-87.647717,Auburn Gresham,48743.0,Winter,Night


In [7]:
df = df[['Case Number','Season','Part of the day','Primary Type','Description','Block','Location Description','Arrest','Beat','District','Community Area','Community Area Description','Community Area Population','Latitude','Longitude']] #set a column order
df.head()

,Case Number,Season,Part of the day,Primary Type,Description,Block,Location Description,Arrest,Beat,District,Community Area,Community Area Description,Community Area Population,Latitude,Longitude
0,JC104662,Winter,Night,CRIMINAL DAMAGE,TO VEHICLE,112XX S SACRAMENTO AVE,STREET,False,2211,22,74.0,Mount Greenwood,19093.0,41.689079,-87.696064
1,JC110056,Winter,Night,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,013XX W 72ND ST,OTHER,False,734,7,67.0,West Englewood,35505.0,41.763181,-87.657709
2,JC100043,Winter,Night,CRIMINAL DAMAGE,TO PROPERTY,084XX S SANGAMON ST,APARTMENT,False,613,6,71.0,Auburn Gresham,48743.0,41.740521,-87.647391
3,JC100006,Winter,Night,BATTERY,AGG: HANDS/FIST/FEET NO/MINOR INJURY,018XX S ALLPORT ST,OTHER,True,1233,12,31.0,Lower West Side,35769.0,41.857068,-87.657625
4,JC100031,Winter,Night,BATTERY,DOMESTIC BATTERY SIMPLE,078XX S SANGAMON ST,APARTMENT,False,621,6,71.0,Auburn Gresham,48743.0,41.751914,-87.647717


In [8]:
total = df.dropna(how="any", inplace=False) #clean missing data
total = total.reset_index(drop=True)

# Final Data Table
total.head()

,Case Number,Season,Part of the day,Primary Type,Description,Block,Location Description,Arrest,Beat,District,Community Area,Community Area Description,Community Area Population,Latitude,Longitude
0,JC104662,Winter,Night,CRIMINAL DAMAGE,TO VEHICLE,112XX S SACRAMENTO AVE,STREET,False,2211,22,74.0,Mount Greenwood,19093.0,41.689079,-87.696064
1,JC110056,Winter,Night,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,013XX W 72ND ST,OTHER,False,734,7,67.0,West Englewood,35505.0,41.763181,-87.657709
2,JC100043,Winter,Night,CRIMINAL DAMAGE,TO PROPERTY,084XX S SANGAMON ST,APARTMENT,False,613,6,71.0,Auburn Gresham,48743.0,41.740521,-87.647391
3,JC100006,Winter,Night,BATTERY,AGG: HANDS/FIST/FEET NO/MINOR INJURY,018XX S ALLPORT ST,OTHER,True,1233,12,31.0,Lower West Side,35769.0,41.857068,-87.657625
4,JC100031,Winter,Night,BATTERY,DOMESTIC BATTERY SIMPLE,078XX S SANGAMON ST,APARTMENT,False,621,6,71.0,Auburn Gresham,48743.0,41.751914,-87.647717


In [9]:
total.shape # 1.5% of data were romoved because of missing coordinates

(263424, 15)

In [10]:
# Final Data Table for Homicide

homicide = total[total['Primary Type']=='HOMICIDE']
homicide.head()

,Case Number,Season,Part of the day,Primary Type,Description,Block,Location Description,Arrest,Beat,District,Community Area,Community Area Description,Community Area Population,Latitude,Longitude
314,JB573872,Winter,Afternoon,HOMICIDE,FIRST DEGREE MURDER,002XX W 95TH ST,STREET,False,634,6,49.0,Roseland,44619.0,41.721658,-87.628972
428,JB573562,Winter,Morning,HOMICIDE,FIRST DEGREE MURDER,003XX E 53RD ST,STREET,True,231,2,40.0,Washington Park,11717.0,41.798450,-87.616858
1098,JB572372,Winter,Morning,HOMICIDE,FIRST DEGREE MURDER,066XX S LAFLIN ST,ABANDONED BUILDING,True,725,7,67.0,West Englewood,35505.0,41.773350,-87.661613
3523,JB567847,Winter,Afternoon,HOMICIDE,FIRST DEGREE MURDER,057XX W 64TH PL,HOUSE,True,812,8,64.0,Clearing,23139.0,41.775264,-87.765095
4048,JB566817,Winter,Afternoon,HOMICIDE,FIRST DEGREE MURDER,024XX S WHIPPLE ST,STREET,False,1033,10,30.0,South Lawndale,79288.0,41.847238,-87.701441


In [11]:
homicide.shape 

(589, 15)